In [ ]:
# 1 Vorbereitung: Installiere alle notwendigen Pakete
!pip install -q transformers accelerate datasets peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00


In [ ]:
# 2 Importieren der Bibliotheken
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [ ]:
# 3 Hugging Face Connection
from huggingface_hub import login
login()  # hier deinen HF-Token einfügen # HIER ANPASSEN @Adri
#

In [ ]:
# 4 CSV-Datei einlesen und in Dataset umwandeln -> ANPASSUNG!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# import pyarrow as pa
# import pyarrow.dataset as ds

from google.colab import drive
drive.mount('/content/drive', force_remount=True) # ggf. ohne mount machen !!! # HIER ANPASSEN @Adri
csv_path = "/content/drive/MyDrive/TrainingMistral/risk_data_formatted_CSV.csv" # HIER ANPASSEN @Adri

# Einlesen der CSV-Datei
df = pd.read_csv(csv_path)
# Test
df.head()

dataset = Dataset.from_pandas(df[['prompt', 'response']])

model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Festlegen des Padding-Tokens
tokenizer.pad_token = tokenizer.eos_token

# Umwandeln des DataFrames in ein Dataset
# dataset = Dataset.from_pandas(df)

# GPT
# splits = dataset.train_test_split(test_size=0.1, seed=42)
# train_raw = splits["train"]
# eval_raw  = splits["test"]

# Medium Artikel
# df['text'] = '<s>[INST] ' + df['prompt'] + '[/INST]' + df['response'] + '</s>'
#df = df.drop(['prompt', 'response'], axis=1)
#dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
#dataset = Dataset(pa.Table.from_pandas(df))

Mounted at /content/drive


In [ ]:
# 4a Laden des Tokenizers , Mistral Vorschlag

# Funktion zum Tokenisieren der Daten
def tokenize_function(examples):
    # Tokenisieren der Texte in den Spalten 'input' und 'output'
    inputs = tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples["response"], padding="max_length", truncation=True, max_length=512)
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': outputs['input_ids']
    }

# Tokenisieren des Datensatzes
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Aufteilen des Datensatzes in Trainings- und Evaluationssätze
splits = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_raw = splits["train"]
eval_raw = splits["test"]

In [ ]:
# 4b Tokenisieren nach GPT überarbeitet

def tokenize(example):
    prompt = example["prompt"].strip()
    response = example["response"].strip()

    # Tokenisiere Prompt und Response separat
    prompt_ids = tokenizer(prompt + tokenizer.eos_token, truncation=True, max_length=256)["input_ids"] #truncation und max_length ausstellen?
    response_ids = tokenizer(response + tokenizer.eos_token, truncation=True, max_length=256)["input_ids"]

    # Verbinde beides
    input_ids = prompt_ids + response_ids

    # Labels: Prompt wird maskiert (nicht gelernt), Response wird gelernt
    labels = [-100] * len(prompt_ids) + response_ids

    # Falls zu lang -> abschneiden
    input_ids = input_ids[:512]
    labels = labels[:512]
    attention_mask = [1] * len(input_ids)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_ds = dataset.map(tokenize, batched=False)


In [ ]:
# 4c Tokenisieren nach Kai (von T5 adaptiert?) -> Völlig FALSCHE Ergebnisse, aber immer lauffähiges Programm

def tokenize(example):
    model_input = tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    labels = tokenizer(
        example["response"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_ds = dataset.map(tokenize, batched=False)

In [ ]:
# 4d Tokenisieren Medium Artikel -> Weird?

# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True) # bereits in 4

tokenizer.padding_side = 'right'

# tokenizer.pad_token = tokenizer.eos_token # bereits in 4

tokenizer.add_eos_token = True

tokenizer.bos_token, tokenizer.eos_token

In [ ]:
# 6 BitsAndBytes Config für QLoRA

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # zu False ändern für mehr Genauigkeit?
    bnb_4bit_quant_type="nf4", # richtig? oder standard?
    bnb_4bit_compute_dtype=torch.float16 # ggf. abändern zu torch.bfloat16 -> T4 unterstützt kein bfloat16
)

In [ ]:
# 7 Modell laden und vorbereiten

model_id = "mistralai/Mistral-7B-v0.1"  # Beispiel-Checkpoint

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.bfloat16, # -> im Medium Artikel
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model.config.use_cache = False # siehe Medium Artikel -> False, um Gradient Checkpointing zu verwenden
model.config.pretraining_tp = 1 # evtl. noch hinzufügen, im Medium Artikel -> Tensor Parallelism -> verteilt auf mehrere GPUs, falls vorhanden
model.gradient_checkpointing_enable() # siehe Medium Artikel
# model.enable_input_require_grads()  # benötigt für QLoRA Training sagt GPT?

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# 8 LoRA-Konfiguration
model = prepare_model_for_kbit_training(model) # Hier einfügen -> siehe Medium Artikel !!!


lora_config = LoraConfig(
    r=16, # je höher, desto höhere Anpassungsfähigkeit/ Parameter, aber auch Overfitting Risiko
    lora_alpha=32, # im richtigen Verhältnis zu r wählen
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], # k_proj und o_proj entfernen?
    lora_dropout=0.1, # Vermeidung von Overfitting -> je höher, desto weniger Overfitting, aber potenziell weniger präzises Lernen
    bias="none",
    task_type="CAUSAL_LM",
)


model = get_peft_model(model, lora_config)

In [ ]:
# 9 TrainingArguments definieren -> ANPASSEN!!!
training_args = TrainingArguments(
    output_dir="qlora-mistral7b",          # Ordner für Checkpoints
    per_device_train_batch_size=4,         # pro GPU
    gradient_accumulation_steps=8,         # effektive Batch = 4×8 = 32
    learning_rate=1e-4,                    # Start-LR # Empfehlung von Mistral
    fp16=True,                             # mixed precision # Falls torch.float16 zu fp16 abändern!!!
    num_train_epochs=3,                    # Epochen
    logging_steps=10,                      # Log alle 10 Schritte
    # eval_strategy="epoch",               # eval nach jeder Epoche
    save_strategy="epoch",                 # speichern nach jeder Epoche
    save_total_limit=2,                    # max. 2 Checkpoints
    report_to="none",                      # verhindert W&B Fehlermeldung
)

In [ ]:
# 10 Trainer initialisieren -> EVTL. SFTTrainer VERWENDEN !!!
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    # peft_config=lora_config, # Medium Artikel
    # max_seq_length= None, # Medium Artikel
    # dataset_text_field="text", # Medium Artikel
    tokenizer=tokenizer,
    args=training_args
    # packing= False, # Medium Artikel
)

<ipython-input-17-f5c99a738d18>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# 11 Training starten
trainer.train()

# wandb API: 01eb4595fbdd612f9d49f834b0d2951f56f7139f

ValueError: No columns in the dataset match the model's forward method signature: ({', '.join(signature_columns)}). The following columns have been ignored: [text]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
# 12 Adapter und Tokenizer speichern
model.save_pretrained("mistral7b-qlora-adapter")
tokenizer.save_pretrained("mistral7b-qlora-adapter")

# statt in /content/ zu speichern, direkt in dein Drive:
model.save_pretrained("/content/drive/MyDrive/DeinOrdner/mistral7b-qlora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/DeinOrdner/mistral7b-qlora-adapter")